# Hands-on RAG Exercise

This notebook provides a step-by-step hands-on exercise for students to build their own RAG pipeline.

## Objectives
- Load and chunk a custom document
- Build a FAISS index with embeddings
- Query the index and return context
- Generate answers using context + language model

In [ ]:
# Install needed packages
%pip install faiss-cpu sentence-transformers openai -q

## Step 1: Load Your Custom Text
Paste or upload your own document below.

In [ ]:
custom_text = """Paste your own text here about any topic (science, history, tech, etc.).
Make sure it's at least a few paragraphs long for meaningful chunking."""

## Step 2: Chunk and Embed

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Simple chunking
chunks = [s.strip() for s in custom_text.split('.') if s]
chunks = [c for c in chunks if len(c) > 20]

model = SentenceTransformer('all-MiniLM-L6-v2')
chunk_vectors = model.encode(chunks)

index = faiss.IndexFlatL2(chunk_vectors.shape[1])
index.add(np.array(chunk_vectors).astype("float32"))

## Step 3: Query Your Index

In [ ]:
query = "Your question here"
query_vec = model.encode([query])

D, I = index.search(np.array(query_vec).astype("float32"), k=3)
retrieved = [chunks[i] for i in I[0]]

print("Retrieved Chunks:")
for r in retrieved:
    print("-", r)

## Step 4: Answer Using Retrieved Context

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

context = "\n".join(retrieved)
prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=100,
    temperature=0.3
)

print("Answer:", response['choices'][0]['text'].strip())

## Summary
You've now:
- Built a searchable knowledge base
- Retrieved context with FAISS
- Used OpenAI to generate an answer grounded in your document